In [6]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [7]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import matplotlib.pyplot as plt
import numpy as np
import os

In [8]:
dados =pd.read_csv('databasePJF.csv', header=0, sep=',', quotechar='"')

In [9]:
dados.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [10]:
#criando um dataframe com menos não fraudes para tornar maior a proporção de fraudes para não fraudes.
dadosfraud = dados[(dados.isFraud == 1)]
dadosnfraud = dados[(dados.isFraud == 0)]
dadosnfraud = dadosnfraud.sample(frac = 1)
dadosnfraud = dadosnfraud.iloc[0:25000]
dadosfinais = pd.concat([dadosfraud,dadosnfraud])
dadosfinais

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
251,1,TRANSFER,2806.00,C1420196421,2806.00,0.00,C972765878,0.00,0.00,1,0
252,1,CASH_OUT,2806.00,C2101527076,2806.00,0.00,C1007251739,26202.00,0.00,1,0
680,1,TRANSFER,20128.00,C137533655,20128.00,0.00,C1848415041,0.00,0.00,1,0
681,1,CASH_OUT,20128.00,C1118430673,20128.00,0.00,C339924917,6268.00,12145.85,1,0
724,1,CASH_OUT,416001.33,C749981943,0.00,0.00,C667346055,102.00,9291619.62,1,0
969,1,TRANSFER,1277212.77,C1334405552,1277212.77,0.00,C431687661,0.00,0.00,1,0
970,1,CASH_OUT,1277212.77,C467632528,1277212.77,0.00,C716083600,0.00,2444985.19,1,0
1115,1,TRANSFER,35063.63,C1364127192,35063.63,0.00,C1136419747,0.00,0.00,1,0


In [11]:
dadosfinais = dadosfinais.sample(frac = 1)
dadosfinais = dadosfinais.reset_index()
dadosfinais.isFraud.value_counts(True)

0    0.752717
1    0.247283
Name: isFraud, dtype: float64

In [12]:
dadosfinais.columns

Index(['index', 'step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg',
       'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest',
       'isFraud', 'isFlaggedFraud'],
      dtype='object')

In [26]:
#utilizando o naive bayes retirado do site https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/
#utilizando a mesma base de dados utilizada para treinar o naive bayes.
dadosteste = dadosfinais[:5000]
dadostreinamento = dadosfinais[5000:]
y = dadostreinamento.isFraud
x = dadostreinamento.loc[:, [ 'step', 'amount', 'oldbalanceOrg',
       'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']]
z = dadosteste.loc[:, [ 'step', 'amount', 'oldbalanceOrg',
       'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']]
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x,y)
predicted = model.predict(z)
print(predicted)
#help(model.predict)


[0 0 0 ..., 0 0 0]


In [28]:
#comparando o real com a classficação do naive bayes
dadosteste['predicted'] = pd.Series(predicted)
dadosteste = dadosteste.loc[:, ["isFraud", "predicted"]]
print(dadosteste.isFraud.value_counts())

print(dadosteste.predicted.value_counts())
dadosteste.head()

0    3757
1    1243
Name: isFraud, dtype: int64
0    4612
1     388
Name: predicted, dtype: int64


,isFraud,predicted
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0


In [40]:
br = []

for a in range(len(dadosteste)):
    
    if dadosteste.isFraud[a] == 1:
        
        if dadosteste.predicted[a] == 1:
            br.append('positivos verdadeiros')
            
        if dadosteste.predicted[a] == 0:
            br.append('negativos falsos')
            
    elif dadosteste.isFraud[a] == 0:
        
        if dadosteste.predicted[a] == 0:
            br.append('negativos verdadeiros')
            
        if dadosteste.predicted[a] == 1:
            br.append('positivos falsos')


 fazer falsos positivos, verdadeiros positivos... (proj2)

In [43]:
##
porcentagem = pd.Series(br)
hue= (porcentagem.value_counts(True))*100
hue

negativos verdadeiros    74.42
negativos falsos         17.82
positivos verdadeiros     7.04
positivos falsos          0.72
dtype: float64

rodar o naive bayes qualitativo (multinomial) e